# Web Scrapping Transfermarket

Coleta dos dados de jogos do Campeonato Brasileiro de Futebol, Série A e Série B, na era dos pontos corridos disponíveis no site Transfermarket:
* Brasileiro Série A (2003 - 2023)
* Brasileiro Série B (2008 - 2023)

### Estrutura dos dados


Os jogos serão armazenados em um Data Frame do Pandas com 11 colunas:
* **date:** dd/mm/aa
* **time:** hh:mm no horário de Brasília
* **home_team:** time mandante
* **away_team:** time visitante
* **goals_home:** gols marcados pelo time mandante
* **goals_away:** gols marcados pelo time visitante
* **referee:** ábitro principal, se a informação estiver disponível
* **attendance:** público presente, se a informação estiver disponível
* **league:** campeonato, 'Serie A' ou 'Serie B'
* **season:** temporada
* **matchweek:** rodada


Gols, cartões vermelhos que ocoorem em uma partida são armazenados em outro Data Frame Pandas:
* **match_index:** contém o índice da linha do Data Frame de jogos a qual a informação está relacionada 
* **event_team:** 'h' se o evento estiver relacionado ao time mandante ou 'a' if se estiver relacionado ao time visitante
* **event_time:** tempo do jogo de 1' até 90'
* **event_player:** nome do jogador relacionado ao evento
* **event_desc:** descrição do evento

### Definição das funções

Dependências utilizadas

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

A partir da informação do campeonato, temporada e rodada, a função retorno o html referente a página com a lista de jogos.

In [2]:
def get_page(league, season, matchweek):
    # define url 
    url_a = 'https://www.transfermarkt.com.br/campeonato-brasileiro-serie-a/spieltag/wettbewerb/BRA1/plus/?saison_id={}&spieltag={}'
    url_b = 'https://www.transfermarkt.com.br/campeonato-brasileiro-serie-b/spieltag/wettbewerb/BRA2/plus/?saison_id={}&spieltag={}'
    if league == 'Serie A':
        url = url_a.format(season-1, matchweek)
    elif league == 'Serie B':
        url = url_b.format(season-1, matchweek)
    # get request
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    # html parser
    page = BeautifulSoup(response.text, "html.parser")
    return page

A função extrai uma lista de tags tbody que contém informação de um jogo

In [3]:
def get_matches_list(page):
    matches_columns = page.select_one('html > body > div > main > div.row > div.large-8.columns')
    matches_list = matches_columns.select('div[style="border-top: 0 !important;"] > table > tbody')
    return matches_list

As funções seguintes extraem informações de uma determinada partida. É realizado um tratamento de erro em cada função com a finalidade de não parar a execução do programa caso uma página possua um construção diferente daquelas para o qual o codigo foi projetado. Nesses casos, o campo será preenchido com "error" no Data Frame para possibilitar o tratamentos desses dados em uma etapa futura.

A função identifica as tags tr dentro da tag tbody que não possuem o parâmetro class definido. A primeira dessas tags contém informação de data e hora. Dentro dessa tag existe uma tag a que possui em seu texto a data da partida. É realizado um tratamento nessa string para retirada de espaçamentos. Caso a informação não esteja disponível, será retornada uma string vazia. 

In [4]:
def get_match_date(match):
    # date
    try:
        match_informations = match.find_all('tr', attrs={'class': None})
        date_time = match_informations[0]
        date = date_time.select_one('td > a').text
        date = date.replace(' ','').replace('\n','')
    except:
        date = "error"
    return date

A função identifica as tags tr dentro da tag tbody que não possuem o parâmetro class definido. A primeira dessas tags contém informação de data e hora. A função encontra uma tag td e retira as tags filhas. O texto que restar dentro da tag td será a informação da hora da partida. É realizada uma limpeza da string para retirada de espaçamentos e caracteres indesejados. Caso a informação não esteja disponível, será retornada uma string vazia.

In [5]:
def get_match_time(match):
    # time
    try:
        match_informations = match.find_all('tr', attrs={'class': None})
        date_time = match_informations[0]
        time = date_time.find('td')
        time.div.decompose()
        time.a.decompose()
        time = re.sub('\s', '', time.text).strip('-')
    except:
        time = "error"
    return time

Dentro da tag tbody, a função identifica a primeira tag tr, a que contem o placar da partida. A partir da tag tr, a função encontra duas tags a dentro de uma tag com uma classe específica. Essas tags a contém no texto o nome das equipes. A primeira será referente à equipe mandante e a segunda referente à equipe visitante.

In [6]:
def get_match_teams(match):
    try:
        # teams
        score = match.find_all('tr')[0]
        teams = score.select('.hide-for-small.spieltagsansicht-vereinsname > a')
        home_team = teams[0].text.strip()
        away_team = teams[1].text.strip()
        team = {'home': home_team, 'away': away_team}
    except:
        team = {'home': "error", 'away': "error"}
    return team

Dentro da tag tbody, a função identifica a primeira tag tr, a que contem o placar da partida. A partir da tag tr, a função encontra uma tag span dentro de uma seguência a partir de uma tag com uma classe específica. Essas tag span contém no texto os gols da partida no formato mm:vv. A string é tratada para a identificação do gol de cada equipe.

In [7]:
def get_match_goals(match):
    try:
        # goals
        score = match.find_all('tr')[0]
        goals = score.select_one('.spieltagsansicht-ergebnis > span > a > span').text
        goals = goals.split(':')
        goals_home = goals[0].strip()
        goals_away = goals[1].strip()
        goals = {'home': goals_home, 'away': goals_away}
    except:
        goals = goals = {'home': "error", 'away': "error"}
    return goals

Em algumas partidas existe a informação referente ao público presente e ao árbitro principal. Nesse caso haverá duas tags tr dentro da tag tbody que não possuem o parâmetro class definido. A segunda dessas tag contém tais informações.

A função identifica as tags tr dentro da tag tbody que não possuem o parâmetro class definido. Caso só exita uma tag, a função retornará None. Caso haja uma segunda tag, a função encontrará o texto dela e filtrará somente os caracteres que forem algarismos. O retorno poderá ser uma string de algarismos representando o público presente ou uma string vazia caso a informação não seja encontrada.

In [8]:
def get_match_attendance(match):
    try:
        # attendance
        match_informations = match.find_all('tr', attrs={'class': None})
        attendance_referee = match_informations[1] if len(match_informations)>1 else None
        if attendance_referee:
            attendance = attendance_referee.text
            attendance = re.sub('[^0-9]', '', attendance)
        else:
            attendance = None
    except:
        attendance = "error"
    return attendance

A função identifica as tags tr dentro da tag tbody que não possuem o parâmetro class definido. Caso só exita uma tag, a função retornará None. Caso haja uma segunda tag, a função selecionará uma tag a. Caso não exista essa tag ou seu conteúdo seja vazio, a função retornará None, caso contrário, o retorno será uma sting contendo o nome do árbitro principal.

In [9]:
def get_match_referee(match):
    try:
        # attendance
        match_informations = match.find_all('tr', attrs={'class': None})
        attendance_referee = match_informations[1] if len(match_informations)>1 else None
        referee = attendance_referee.select_one('a') if attendance_referee else None
        referee = referee.text if referee else None
    except:
        referee = "error"
    return referee

Em algumas temporadas, estão disponíveis informações de alguns eventos que ocorreram no jogo: 'Gol', 'Gol de pênalti', 'Cartão vermelho', 'Gol-contra', 'Segundo cartão amarelo', 'Pênalti perdido'.

A função procura na página elementos que possuem a classe que os determina como atributo. Caso nenhuma tag seja encontrada, a função retorna uma lista vazia.

In [10]:
def get_match_events(match):
    try:
        match_events = match.select('tr.no-border.spieltagsansicht-aktionen')
    except:
        match_events = []
    return match_events

A função encontra as tags td que descrevem o evento. Na sequência encontra a tag que possui uma sequencia de elementos que contem o evento. A partir da posição dessa tag, é possível determinar as características do evento. A função retorna uma lista contendo as informações e adicionado como primeiro elemento, o indice da partida no Data Frame das partidas a que jogo se refere o evento.

In [11]:
def get_event_data(event, match_index):
    try:
        event = event.find_all('td')
        event_index = [i for (i, td) in enumerate(event) if td.select_one('div > div > span > a')][0]
        event_team = 'h' if event_index==0 else 'a'
        event_time = event[1] if event_team=='h' else event[3]
        event_time = event_time.text
        event_desc = event[event_index].select_one('span.icons_sprite')
        if event_desc:
            event_desc = event_desc['title']
            event_desc = event_desc.split(':')[1].strip()
        else:
            event_desc = None
        event_player = event[event_index].select_one('div > div > span > a')['title']
        event = [match_index, event_team, event_time, event_player, event_desc]
    except:
        event = [match_index, None, None, None, None]
    return event

### Execução do Script

In [12]:
# matches DataFrame
matches_df = pd.DataFrame(columns=[
    'date','time','home_team','away_team','goals_home','goals_away','referee','attendance','league','season','matchweek'
])
matches_df.index.name = 'match'

# events DataFrame
events_df = pd.DataFrame(columns=['match', 'team', 'time', 'player', 'desc'])

# Serie A

tm_site_list = []

league = 'Serie A'
for season in range(2003, 2005):
    for matchweek in range(1,47):
        tm_site = {}
        tm_site['league'] = league
        tm_site['season'] = season
        tm_site['matchweek'] = matchweek
        tm_site_list.append(tm_site)
for season in range(2005, 2006):
    for matchweek in range(1,43):
        tm_site = {}
        tm_site['league'] = league
        tm_site['season'] = season
        tm_site['matchweek'] = matchweek
        tm_site_list.append(tm_site)
for season in range(2006, 2024):
    for matchweek in range(1,39):
        tm_site = {}
        tm_site['league'] = league
        tm_site['season'] = season
        tm_site['matchweek'] = matchweek
        tm_site_list.append(tm_site)
        
league = 'Serie B'
for season in range(2008, 2024):
    for matchweek in range(1,47):
        tm_site = {}
        tm_site['league'] = league
        tm_site['season'] = season
        tm_site['matchweek'] = matchweek
        tm_site_list.append(tm_site)

        
for tm_site in tm_site_list:
    league, season, matchweek = tm_site['league'], tm_site['season'], tm_site['matchweek']
    print(f'{league} {season} - Matchweek {matchweek}')
    page = get_page(league, season, matchweek)
    matches_list = get_matches_list(page)
    
    for match in matches_list:
        date = get_match_date(match)
        time = get_match_time(match)
        team = get_match_teams(match)
        goals = get_match_goals(match)
        attendance = get_match_attendance(match)
        referee = get_match_referee(match)
        match_events = get_match_events(match)
        # to matche dataframe
        match_row = [
            date, time, team['home'], team['away'], goals['home'], goals['away'], referee, attendance, league, season, matchweek
        ]
        match_index = matches_df.shape[0]
        matches_df.loc[match_index] = match_row 
        
        for event in match_events:
            event_row = get_event_data(event, match_index)
            # to event dataframe
            events_df.loc[events_df.shape[0]] = event_row
        

Serie A 2003 - Matchweek 1
Serie A 2003 - Matchweek 2
Serie A 2003 - Matchweek 3
Serie A 2003 - Matchweek 4
Serie A 2003 - Matchweek 5
Serie A 2003 - Matchweek 6
Serie A 2003 - Matchweek 7
Serie A 2003 - Matchweek 8
Serie A 2003 - Matchweek 9
Serie A 2003 - Matchweek 10
Serie A 2003 - Matchweek 11
Serie A 2003 - Matchweek 12
Serie A 2003 - Matchweek 13
Serie A 2003 - Matchweek 14
Serie A 2003 - Matchweek 15
Serie A 2003 - Matchweek 16
Serie A 2003 - Matchweek 17
Serie A 2003 - Matchweek 18
Serie A 2003 - Matchweek 19
Serie A 2003 - Matchweek 20
Serie A 2003 - Matchweek 21
Serie A 2003 - Matchweek 22
Serie A 2003 - Matchweek 23
Serie A 2003 - Matchweek 24
Serie A 2003 - Matchweek 25
Serie A 2003 - Matchweek 26
Serie A 2003 - Matchweek 27
Serie A 2003 - Matchweek 28
Serie A 2003 - Matchweek 29
Serie A 2003 - Matchweek 30
Serie A 2003 - Matchweek 31
Serie A 2003 - Matchweek 32
Serie A 2003 - Matchweek 33
Serie A 2003 - Matchweek 34
Serie A 2003 - Matchweek 35
Serie A 2003 - Matchweek 36
S

Serie A 2010 - Matchweek 11
Serie A 2010 - Matchweek 12
Serie A 2010 - Matchweek 13
Serie A 2010 - Matchweek 14
Serie A 2010 - Matchweek 15
Serie A 2010 - Matchweek 16
Serie A 2010 - Matchweek 17
Serie A 2010 - Matchweek 18
Serie A 2010 - Matchweek 19
Serie A 2010 - Matchweek 20
Serie A 2010 - Matchweek 21
Serie A 2010 - Matchweek 22
Serie A 2010 - Matchweek 23
Serie A 2010 - Matchweek 24
Serie A 2010 - Matchweek 25
Serie A 2010 - Matchweek 26
Serie A 2010 - Matchweek 27
Serie A 2010 - Matchweek 28
Serie A 2010 - Matchweek 29
Serie A 2010 - Matchweek 30
Serie A 2010 - Matchweek 31
Serie A 2010 - Matchweek 32
Serie A 2010 - Matchweek 33
Serie A 2010 - Matchweek 34
Serie A 2010 - Matchweek 35
Serie A 2010 - Matchweek 36
Serie A 2010 - Matchweek 37
Serie A 2010 - Matchweek 38
Serie A 2011 - Matchweek 1
Serie A 2011 - Matchweek 2
Serie A 2011 - Matchweek 3
Serie A 2011 - Matchweek 4
Serie A 2011 - Matchweek 5
Serie A 2011 - Matchweek 6
Serie A 2011 - Matchweek 7
Serie A 2011 - Matchweek 8


Serie A 2018 - Matchweek 2
Serie A 2018 - Matchweek 3
Serie A 2018 - Matchweek 4
Serie A 2018 - Matchweek 5
Serie A 2018 - Matchweek 6
Serie A 2018 - Matchweek 7
Serie A 2018 - Matchweek 8
Serie A 2018 - Matchweek 9
Serie A 2018 - Matchweek 10
Serie A 2018 - Matchweek 11
Serie A 2018 - Matchweek 12
Serie A 2018 - Matchweek 13
Serie A 2018 - Matchweek 14
Serie A 2018 - Matchweek 15
Serie A 2018 - Matchweek 16
Serie A 2018 - Matchweek 17
Serie A 2018 - Matchweek 18
Serie A 2018 - Matchweek 19
Serie A 2018 - Matchweek 20
Serie A 2018 - Matchweek 21
Serie A 2018 - Matchweek 22
Serie A 2018 - Matchweek 23
Serie A 2018 - Matchweek 24
Serie A 2018 - Matchweek 25
Serie A 2018 - Matchweek 26
Serie A 2018 - Matchweek 27
Serie A 2018 - Matchweek 28
Serie A 2018 - Matchweek 29
Serie A 2018 - Matchweek 30
Serie A 2018 - Matchweek 31
Serie A 2018 - Matchweek 32
Serie A 2018 - Matchweek 33
Serie A 2018 - Matchweek 34
Serie A 2018 - Matchweek 35
Serie A 2018 - Matchweek 36
Serie A 2018 - Matchweek 37


Serie B 2009 - Matchweek 24
Serie B 2009 - Matchweek 25
Serie B 2009 - Matchweek 26
Serie B 2009 - Matchweek 27
Serie B 2009 - Matchweek 28
Serie B 2009 - Matchweek 29
Serie B 2009 - Matchweek 30
Serie B 2009 - Matchweek 31
Serie B 2009 - Matchweek 32
Serie B 2009 - Matchweek 33
Serie B 2009 - Matchweek 34
Serie B 2009 - Matchweek 35
Serie B 2009 - Matchweek 36
Serie B 2009 - Matchweek 37
Serie B 2009 - Matchweek 38
Serie B 2009 - Matchweek 39
Serie B 2009 - Matchweek 40
Serie B 2009 - Matchweek 41
Serie B 2009 - Matchweek 42
Serie B 2009 - Matchweek 43
Serie B 2009 - Matchweek 44
Serie B 2009 - Matchweek 45
Serie B 2009 - Matchweek 46
Serie B 2010 - Matchweek 1
Serie B 2010 - Matchweek 2
Serie B 2010 - Matchweek 3
Serie B 2010 - Matchweek 4
Serie B 2010 - Matchweek 5
Serie B 2010 - Matchweek 6
Serie B 2010 - Matchweek 7
Serie B 2010 - Matchweek 8
Serie B 2010 - Matchweek 9
Serie B 2010 - Matchweek 10
Serie B 2010 - Matchweek 11
Serie B 2010 - Matchweek 12
Serie B 2010 - Matchweek 13
S

Serie B 2015 - Matchweek 43
Serie B 2015 - Matchweek 44
Serie B 2015 - Matchweek 45
Serie B 2015 - Matchweek 46
Serie B 2016 - Matchweek 1
Serie B 2016 - Matchweek 2
Serie B 2016 - Matchweek 3
Serie B 2016 - Matchweek 4
Serie B 2016 - Matchweek 5
Serie B 2016 - Matchweek 6
Serie B 2016 - Matchweek 7
Serie B 2016 - Matchweek 8
Serie B 2016 - Matchweek 9
Serie B 2016 - Matchweek 10
Serie B 2016 - Matchweek 11
Serie B 2016 - Matchweek 12
Serie B 2016 - Matchweek 13
Serie B 2016 - Matchweek 14
Serie B 2016 - Matchweek 15
Serie B 2016 - Matchweek 16
Serie B 2016 - Matchweek 17
Serie B 2016 - Matchweek 18
Serie B 2016 - Matchweek 19
Serie B 2016 - Matchweek 20
Serie B 2016 - Matchweek 21
Serie B 2016 - Matchweek 22
Serie B 2016 - Matchweek 23
Serie B 2016 - Matchweek 24
Serie B 2016 - Matchweek 25
Serie B 2016 - Matchweek 26
Serie B 2016 - Matchweek 27
Serie B 2016 - Matchweek 28
Serie B 2016 - Matchweek 29
Serie B 2016 - Matchweek 30
Serie B 2016 - Matchweek 31
Serie B 2016 - Matchweek 32
S

Serie B 2022 - Matchweek 16
Serie B 2022 - Matchweek 17
Serie B 2022 - Matchweek 18
Serie B 2022 - Matchweek 19
Serie B 2022 - Matchweek 20
Serie B 2022 - Matchweek 21
Serie B 2022 - Matchweek 22
Serie B 2022 - Matchweek 23
Serie B 2022 - Matchweek 24
Serie B 2022 - Matchweek 25
Serie B 2022 - Matchweek 26
Serie B 2022 - Matchweek 27
Serie B 2022 - Matchweek 28
Serie B 2022 - Matchweek 29
Serie B 2022 - Matchweek 30
Serie B 2022 - Matchweek 31
Serie B 2022 - Matchweek 32
Serie B 2022 - Matchweek 33
Serie B 2022 - Matchweek 34
Serie B 2022 - Matchweek 35
Serie B 2022 - Matchweek 36
Serie B 2022 - Matchweek 37
Serie B 2022 - Matchweek 38
Serie B 2022 - Matchweek 39
Serie B 2022 - Matchweek 40
Serie B 2022 - Matchweek 41
Serie B 2022 - Matchweek 42
Serie B 2022 - Matchweek 43
Serie B 2022 - Matchweek 44
Serie B 2022 - Matchweek 45
Serie B 2022 - Matchweek 46
Serie B 2023 - Matchweek 1
Serie B 2023 - Matchweek 2
Serie B 2023 - Matchweek 3
Serie B 2023 - Matchweek 4
Serie B 2023 - Matchweek

Gravação dos data frames em arquivos csv

In [13]:
matches_df.to_csv('tm_matches.csv')
events_df.to_csv('tm_events.csv')